# Exercise 0.1
## Introduction to EM data, neuron segmentation and gunpowder
### A simple gunpowder example pipeline for loading and manipulating data

Become familiar with gunpowder. Read and try to understand the following code as well as the general principle behind gunpowder. Read the introductory example in the gunpowder documentation: https://github.com/funkey/gunpowder/

In [2]:
from __future__ import print_function
from gunpowder import *
from gunpowder.tensorflow import *
import numpy as np
import os


# A simple gunpowder pipeline for loading and manipulating data:
data_dir = '../../jan/segmentation/data'

samples = [
    'sample_A',
    'sample_B',
    'sample_C'
]


# Define gunpowder variables of interest:
raw = ArrayKey('RAW') # Raw EM data
labels = ArrayKey('GT_LABELS') # ground truth neuron segmentation 
affinities = ArrayKey('GT_AFFINITIES') # affinities

# Voxel size is the physical size of one voxel (=3D pixel) in nm.
voxel_size = Coordinate((40, 4, 4))
batch_size = Coordinate((30,1000,1000)) * voxel_size

# Request all the data you need for training:
request = BatchRequest()
request.add(raw, batch_size)
request.add(labels, batch_size)
request.add(affinities, batch_size)

# Request a snapshot s.t. you are able to visualize the data in your pipeline.
snapshot_request = BatchRequest({
    raw: request[raw],
    labels: request[labels],
    affinities: request[affinities]
})

# Note that the data only provides raw and neuron_ids which is the neuron segmentation.
# However, we need affinities which we can generate from neuron_ids by using gunpowder (see below):
data_sources = tuple(
    N5Source(
        os.path.join(data_dir, sample + '.n5'),
        datasets = {
            raw: 'volumes/raw',
            labels: 'volumes/labels/neuron_ids',
        },
        array_specs = {
            raw: ArraySpec(interpolatable=True),
            labels: ArraySpec(interpolatable=False)
        }
    ) +
    Normalize(raw) + 
    #Pad(labels, None) +
    RandomLocation()
    for sample in samples
    )


# Define a neighborhood for affinities:
neighborhood = [[-1, 0, 0], [0, -1, 0], [0, 0, -1]]

pipeline = (
    data_sources +
    RandomProvider() +
    AddAffinities(neighborhood,
                 labels=labels,
                 affinities=affinities) +
    Snapshot({raw: 'volumes/raw',
              labels: 'volumes/labels/neuron_ids',
              affinities: 'volumes/affinities'}))
    
    
with build(pipeline) as b:
    b.request_batch(request)

# Exercise 0.2 
## Visualize data using neuroglancer

Exercise 0.1 generated a file in your working directory containg the raw EM data, neuron segmentation and affinities. The following script lets you view the data. Run both cells below and click on the resulting link. You can enable and disable layers by clicking on them. Explore the data. What do the different colors of the affinities mean? How does it relate to the segmentation?

If you double click on a particular segment neuroglancer allows you to view the 3D mesh of the object.

In [4]:
import neuroglancer
import funlib.show.neuroglancer as funlibng
import daisy
import numpy as np

# Replace my_ip with your public paperspace ip as shown in your browser
my_ip = "172.83.14.204"
neuroglancer.set_server_bind_address('0.0.0.0', 8889)

data_file = './snapshots/00000000.hdf'
affs_ds = daisy.open_ds(data_file, 'volumes/affinities')
affs_ds.data = np.array(affs_ds.data, dtype=np.float32)
labels_ds = daisy.open_ds(data_file, 'volumes/labels/neuron_ids')
raw_ds = daisy.open_ds(data_file, 'volumes/raw')

neuroglancer.set_server_bind_address('0.0.0.0')
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    funlibng.add_layer(s, raw_ds, 'raw')
    funlibng.add_layer(s, labels_ds, 'labels')
    funlibng.add_layer(s, affs_ds, 'affinities', shader='rgb')

print(viewer.__str__().replace("localhost", my_ip))

http://172.83.14.204:42551/v/cdffd852c880fef9568ebc2a30e1e67ca28845b1/


# Exercise 1
## Using gunpowder with tensorflow for training a 3D-UNet for affinity prediction
#### mknet.py

In this exercise we ask you to fill in the given template file to create a computation graph for your neural network. Follow the instructions in the code and make use of help and documentations. 

As usual if you are not sure about what a function does or what its argument is navigate to the function and press ```shift + tab``` to display arguments and documentation.

In [2]:
from funlib.learn.tensorflow import models
import malis
import tensorflow as tf
import json

def create_network(input_shape, name):

    tf.reset_default_graph()

    with tf.variable_scope('setup0'):

        raw = tf.placeholder(tf.float32, shape=input_shape)
        raw_batched = tf.reshape(raw, (1, 1) + input_shape)
        
        """
        Define your model architecture here:
        
        1. Set the main model parameters for the UNet:
        
        unet, _, _ = models.unet(...)
        
        2. Generate the desired number of output features (3 for affinities in z,y,x direction)
           by using a convolution with kernel size 1:
           
        affs_batched = models.conv_pass(unet, ...)
        """

        output_shape_batched = affs_batched.get_shape().as_list()
        output_shape = output_shape_batched[1:] # strip the batch dimension

        affs = tf.reshape(affs_batched, output_shape)

        gt_affs = tf.placeholder(tf.float32, shape=output_shape)
        affs_loss_weights = tf.placeholder(tf.float32, shape=output_shape)
        
        """
        Define your loss here. For reference see tensorflow.losses documentation.
        
        loss = ...
        """
        
        loss = tf.losses.mean_squared_error(
            gt_affs,
            affs,
            affs_loss_weights)

        # Generate a summary for tensorboard:
        summary = tf.summary.scalar('setup0', loss)

        """
        Choose an optimizer and learning parameters. Minimize the loss.
        For reference see tensorflow.train
        
        opt = ...
        optimizer = opt.minimize(loss)
        """

        output_shape = output_shape[1:]
        print("input shape : %s"%(input_shape,))
        print("output shape: %s"%(output_shape,))

        # Export your computation graph:
        tf.train.export_meta_graph(filename=name + '.meta')

        # Write out the names of relevant tensors in this graph s.t. the train script can feed and receive values.
        config = {
            'raw': raw.name,
            'affs': affs.name,
            'gt_affs': gt_affs.name,
            'affs_loss_weights': affs_loss_weights.name,
            'loss': loss.name,
            'optimizer': optimizer.name,
            'input_shape': input_shape,
            'output_shape': output_shape,
            'summary': summary.name
        }

        config['outputs'] = {'affs': {"out_dims": 3, "out_dtype": "uint8"}}

        with open(name + '.json', 'w') as f:
            json.dump(config, f)

if __name__ == "__main__":
    # Run the script and produce the necessary files for training:
    create_network((84, 268, 268), 'train_net')

Creating U-Net layer 0
f_in: (1, 1, 84, 268, 268)
number of variables added: 4236, new total: 4236
    Creating U-Net layer 1
    f_in: (1, 12, 80, 88, 88)
    number of variables added: 116760, new total: 120996
        Creating U-Net layer 2
        f_in: (1, 60, 76, 28, 28)
        number of variables added: 2916600, new total: 3037596
            Creating U-Net layer 3
            f_in: (1, 300, 24, 8, 8)
            bottom layer
            f_out: (1, 1500, 20, 4, 4)
            number of variables added: 72903000, new total: 75940596
        g_out: (1, 1500, 20, 4, 4)
        g_out_upsampled: (1, 300, 60, 12, 12)
        f_left_cropped: (1, 300, 60, 12, 12)
        f_right: (1, 600, 60, 12, 12)
        f_out: (1, 300, 56, 8, 8)
        number of variables added: 19440900, new total: 95381496
    g_out: (1, 300, 56, 8, 8)
    g_out_upsampled: (1, 60, 56, 24, 24)
    f_left_cropped: (1, 60, 56, 24, 24)
    f_right: (1, 120, 56, 24, 24)
    f_out: (1, 60, 52, 20, 20)
    number of v

# Exercise 2
## A gunpowder training pipeline

In this exercise we ask you to use gunpowder in combination with tensorflow to build a training pipeline for affinity prediction. Fill in the gaps in the template code below and train your network!

In [ ]:
from __future__ import print_function
import sys
from gunpowder import *
from gunpowder.tensorflow import *
import os
import math
import json
import tensorflow as tf
import numpy as np
import logging

logging.basicConfig(level=logging.INFO)


# Adapt the path to where the training data is stored:
data_dir = '../../jan/segmentation/data'

samples = [
    'sample_A',
    'sample_B',
    'sample_C'
]

# This defines how we calculate affinities, in this case we consider direct neighbours in z,y,x direction:
neighborhood = [[-1, 0, 0], [0, -1, 0], [0, 0, -1]]

def train_until(max_iteration):

    if tf.train.latest_checkpoint('.'):
        trained_until = int(tf.train.latest_checkpoint('.').split('_')[-1])
    else:
        trained_until = 0
    if trained_until >= max_iteration:
        return

    # Load tensor names from the file you generated above:
    with open('train_net.json', 'r') as f:
        config = json.load(f)

    # Define gunpowder variables of interest
    raw = ArrayKey('RAW')
    labels = ArrayKey('GT_LABELS')
    labels_mask = ArrayKey('GT_LABELS_MASK')
    affs = ArrayKey('PREDICTED_AFFS')
    gt = ArrayKey('GT_AFFINITIES')
    gt_mask = ArrayKey('GT_AFFINITIES_MASK')
    gt_scale = ArrayKey('GT_AFFINITIES_SCALE')
    affs_gradient = ArrayKey('AFFS_GRADIENT')

    voxel_size = Coordinate((40, 4, 4))
    input_size = Coordinate(config['input_shape'])*voxel_size
    output_size = Coordinate(config['output_shape'])*voxel_size
    context = output_size/2
    print('CONTEXT: ', context)

    
    """
    Build the actual data loading and train pipeline by chaining
    gunpowder nodes. Each gunpowder node provides an atomic operation 
    and can request data from upstream nodes. For reference see:
    https://github.com/funkey/gunpowder
    
    In the following we provide you with a skeleton
    pipeline. You should add the actual training node, data preprocessing nodes and 
    data augmentation nodes.
    """
    
    # Request all the data you need for training:
    request = BatchRequest()
    request.add(raw, input_size)
    request.add(labels, output_size)
    request.add(labels_mask, output_size)
    request.add(gt, output_size)
    request.add(gt_mask, output_size)
    request.add(gt_scale, output_size)

    # Request a snapshot s.t. you are able to visualize the data used and produced during training.
    snapshot_request = BatchRequest({
        affs: request[gt],
        affs_gradient: request[gt]
    })

    # Define a data source. In this case the data we provide contains 
    # raw EM images together with neuron segments (neuron_ids)
    # and a mask for regions in the EM data that should not contribute towards 
    # training.
    data_sources = tuple(
        N5Source(
            os.path.join(data_dir, sample + '.n5'),
            datasets = {
                raw: 'volumes/raw',
                labels: 'volumes/labels/neuron_ids',
                labels_mask: 'volumes/labels/mask',
            },
            array_specs = {
                raw: ArraySpec(interpolatable=True),
                labels: ArraySpec(interpolatable=False),
                labels_mask: ArraySpec(interpolatable=False)
            }
        ) +
        Normalize(raw) + 
        Pad(labels, context) +
        Pad(labels_mask, context) +
        RandomLocation() +
        Reject(mask=labels_mask)
        for sample in samples
    )

    """
    Define the actual train pipeline here:
    
    train_pipeline = (
        data_sources +
        RandomProvider() +
        ```Add elastic augmentation``` +
        ```Add simple augmentation``` + 
        ```Add intensity augmentation``` +
        GrowBoundary(labels, labels_mask, steps=1, only_xy=True) +
        ```So far the data sources only provide neuron labels
            and the raw em data. We need to generate the actual training
            data which are affinities. Add a node that does this here.``` +
        BalanceLabels(
            gt,
            gt_scale,
            gt_mask) +
        ```Add defect augment without artifacts```
        IntensityScaleShift(raw, 2,-1) +
        PreCache(cache_size=40,
                 num_workers=10) +
        ```
        Add the actual Train node here:
        Train(...) +
        ```
        IntensityScaleShift(raw, 0.5, 0.5) +
        Snapshot({
                raw: 'volumes/raw',
                labels: 'volumes/labels/neuron_ids',
                gt: 'volumes/gt_affinities',
                affs: 'volumes/pred_affinities',
                gt_mask: 'volumes/labels/gt_mask',
                labels_mask: 'volumes/labels/mask',
                affs_gradient: 'volumes/affs_gradient'
            },
            dataset_dtypes={
                labels: np.uint64
            },
            every=1000,
            output_filename='batch_{iteration}.hdf',
            additional_request=snapshot_request) +
        PrintProfilingStats(every=10)
    )
    
    """

    print("Starting training...")
    """
    Implement the train loop:
    with build(train_pipeline) as b:
        ...
    """
    print("Training finished")

if __name__ == "__main__":
    iteration = 500000
    train_until(iteration)

INFO:gunpowder.tensorflow.local_server:Creating local tensorflow server
INFO:gunpowder.tensorflow.local_server:Server running at b'grpc://localhost:36769'
INFO:gunpowder.tensorflow.nodes.train:Initializing tf session, connecting to b'grpc://localhost:36769'...


CONTEXT:  (960, 112, 112)
Starting training...


INFO:gunpowder.tensorflow.nodes.train:Reading meta-graph...


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.
INFO:gunpowder.tensorflow.nodes.train:No checkpoint found


# Exercise 3

## Monitor the training progress

Visualize the generated training snapshots with neuroglancer and observe loss curves using tensorboard.


#### Snapshots

In [ ]:
import neuroglancer
import funlib.show.neuroglancer as funlibng
import daisy
import numpy as np

"""
Replace the path for the data file with the snapshot you want to visualize. 

data_file = ...
"""

gt_affs_ds = daisy.open_ds(data_file, 'volumes/gt_affinities')
gt_affs_ds.data = np.array(gt_affs_ds.data, dtype=np.float32)

pred_affs_ds = daisy.open_ds(data_file, 'volumes/pred_affinities')
pred_affs_ds.data = np.array(pred_affs_ds.data, dtype=np.float32)

gradients_ds = daisy.open_ds(data_file, 'volumes/affs_gradient')

labels_ds = daisy.open_ds(data_file, 'volumes/labels/neuron_ids')
raw_ds = daisy.open_ds(data_file, 'volumes/raw')

# Replace my_ip with your public paperspace ip as shown in your browser
my_ip = "172.83.14.204"
neuroglancer.set_server_bind_address('0.0.0.0', 8889)

viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    funlibng.add_layer(s, raw_ds, 'raw')
    funlibng.add_layer(s, labels_ds, 'labels')
    funlibng.add_layer(s, gt_affs_ds, 'gt_affinities', shader='rgb')
    funlibng.add_layer(s, pred_affs_ds, 'pred_affinities', shader='rgb')
    funlibng.add_layer(s, gradients_ds, 'gradients')

print(viewer.__str__().replace("localhost", my_ip))

#### Loss
To visualize your loss using tensorboard open a terminal and execute: 
```tensorboard --logdir=path/to/log-directory```